In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install requests beautifulsoup4 newspaper3k reportlab fpdf

In [47]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import os
from fpdf import FPDF

In [48]:
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

pdf.add_page()
pdf.set_font("Arial", size=11)

# Thêm font Unicode
pdf.add_font('DejaVu', '', '/content/drive/MyDrive/crawling/font/DejaVuSans.ttf', uni=True)
pdf.set_font('DejaVu', '', 11)

notebook_path = '/content/drive/MyDrive/crawling/medic'

In [49]:
url = "https://tamanhhospital.vn/benh/"

In [50]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
slug = soup.findAll('a', class_='cl_33 sz_14 lh_12 div_inlineb')
links_and_text = [(a['href'], a.text.strip()) for a in slug if 'href' in a.attrs]

In [ ]:
for link, text in links_and_text:

    print(f"Chủ đề: {text}")
    response = requests.get(link)

    soup = BeautifulSoup(response.content, "html.parser")
    titles = soup.findAll('a', class_='title_post')

    posts = [a['href'] for a in titles if 'href' in a.attrs]

    for post in posts:
      news = requests.get(post)
      sup = BeautifulSoup(news.content, "html.parser")

      title_element = sup.find("h1", class_="cl_seoyc sz_20 font_hel text-uppercase mb_15 aaaa")
      if title_element:
        title = title_element.text.strip()
      else:
        title = "Không tìm thấy tiêu đề"

      body = sup.find("div", id="ftwp-postcontent")
      if body:
        paragraphs = body.find_all("p", recursive=False)
        content = "\n".join([p.text.strip() for p in paragraphs])
      else:
        content = "Không tìm thấy nội dung"

      # print(f"Tiêu đề: {title}")
      # print(f"Nội dung:\n{content}")
      # print("_" * 80)

      pdf.cell(0, 10, txt=f"Tiêu đề: {title}", ln=True)
      pdf.multi_cell(0, 10, txt=f"Nội dung:\n{content}\n")
      pdf.cell(0, 10, txt="_" * 80, ln=True)

    pdf_file_path = os.path.join(notebook_path, f"{text}.pdf")

    pdf.output(pdf_file_path)

    print(f"File PDF đã được lưu tại: {pdf_file_path}")